# Data Manipulation and Cleaning

Data science is a powerful tool for understanding historical events. It allows us to transform raw information, calculate new metrics, summarize data, and deal with imperfections in historical records. Today, we'll learn key **data manipulation and cleaning** techniques by analyzing one of the most famous datasets in the world: the passenger list from the Titanic.

We'll load the **Titanic dataset** directly from a URL. This is a common and efficient way to access public datasets.

In [ ]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
titanic = pd.read_csv(url)
titanic.head()

## Modifying Columns

In [ ]:
# Display the columns
titanic.columns

Often a dataset may not have the most intuitive or descriptive column names. In this case, it is a good idea to provide a "data dicitonary":

| Variable | Definition | Key |
|---|---|---|
| Survived | Survival | 0 = No, 1 = Yes |
| Pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| Name | Name
| Sex | Sex | |
| Age | Age | Age in years |
| SibSp | # of siblings / spouses aboard the Titanic | |
| Parch | # of parents / children aboard the Titanic | |
| Ticket | Ticket number | |
| Fare | Passenger fare | |
| Cabin | Cabin number | |
| Embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |

### Renaming Columns

Dataset manipulation is often about getting the data in a state that is easier to work with. We will be using the column names a lot so it helps to have names that are easier to work with. Here the columns have capital letters. This can create a lot of headaches as our variables are case-sensitive.

In [ ]:
titanic.columns

Since the `.columns` attribute is a list, we can change it by setting a list of new column names to `dataframe.columns`:

In [ ]:
titanic.columns = ['passengerid', 'survived', 'pclass', 'name', 'sex', 'age',
                   'sibsp', 'parch', 'ticket', 'fare', 'cabin', 'embarked']

In [ ]:
titanic.head()

### Adding New Columns

We can often combine some of the columns in some way that yields a new column that creates a variable that will be useful. For example, perhaps we want a column that gives the family size for each passenger. 

Creating new columns in Pandas is pretty straightforward. We just assign it like a variable:

`dataframe['new_column'] = value or calculation`

There are several things you can put in the `value or calculation` part, including conditions, but for now, we will simply add the values of the `sibsp` and `parch` columns for each passenger.

In [ ]:
# We add 1 to count the passenger themselves
titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1
titanic.head()

#### Using Conditional Logic: `np.where()`
This allows you to create values based on conditions. For example, we can categorize passengers as as alone or not using our new `family_size` column:

In [ ]:
titanic['was_alone'] = np.where(titanic['family_size'] == 1, 'Yes', 'No')
titanic.head()

___
## 💪 **Exercise** 💪

1. Add a new column called `age_group` to `titanic`. For simplicity, if a passenger's `age` is less than 18, they should be labeled 'child'; otherwise, they should be labeled 'adult'.
2. The `fare` is in 1912 British Pounds. Let's create a rough estimate for today's value. Create a new column `fare_modern_usd` by multiplying the `Fare` column by 100. Print the head of the updated DataFrame.
---

## Basic Descriptive Statistics for the Titanic Data

Descriptive statistics provide a quick and powerful way to understand the main features of your dataset. By calculating measures like the mean, median, maximum, minimum, and counts, we can get a snapshot of our data's distribution, central tendency, and variability.

In this section, we'll apply these essential statistical tools to our Titanic passenger data. This will help us quickly identify key characteristics, such as the average age of passengers, the range of fares paid, and the distribution of genders on board. These initial patterns can then guide more in-depth analysis.

The columns in a Pandas dataframe are called "Pandas Series". There are many Series methods. Here are the most commonly used:
- `.mean()`: average
- `.max()`: maximum value in the column
- `.counts()`: number of occurences of each unique entry
- `.mode()`: returns the most frequent value(s)

In [ ]:
average_age = titanic['age'].mean()
print(f"Average age of passenger: {average_age:.2f}")   # :.2f rounds to 2 dec. places

In [ ]:
max_fare = titanic['fare'].max()
print(f"Maximum Fare Paid: £{max_fare:.0f}")

In [ ]:
gender_counts = titanic['sex'].value_counts()
print(gender_counts)

In [ ]:
class_mode = titanic['pclass'].mode()
print(class_mode)

Because there can be more than one mode, Pandas returns a Series. Here the `0` is the index for that Series (first entry). The `3` is the actual mode.

___
## 💪 **Exercise** 💪

1. Calculate the survival rate of the passengers, expressed as a percentage. 
2. Find out how many passengers were in each `pclass` (Passenger Class). Print the counts.
___

## Handling Missing Data

Historical data is rarely perfect. The Titanic dataset is no exception. We have already seen a way to detect "null" (missing). Which columns have missing data? 

In [ ]:
titanic.info()

### Identifying Missing Values

Missing values are often represented as `NaN` (Not a Number) in Pandas. A simpler way to count how many missing values are in each column is to use the `.isnull()` function. However, it returns a boolean object that is the same size as the dataframe. Therefore, we can link it to the `.sum()` function to add up the number of missing valies:

In [ ]:
print("Count of missing values per column:")
titanic.isnull().sum()

### Option 1: Dropping Data

Missing data can cause issues with calculations, visualizations, and many data analysis techniques.

The `dropna()` function in Pandas is used to **remove rows or columns that contain missing (NaN) values** from a dataframe. It's a quick way to clean up your data, though you need to be cautious as it can lead to significant data loss if many values are missing. `.dropna()` without any arguments can be rather agressive

In [ ]:
# Drop rows with ANY missing value (most common use)
print(f"Original number of passengers: {len(titanic)}")

titanic_dropped = titanic.dropna()
print(f"Passengers remaining after dropping rows any missing value: {len(titanic_dropped)}")

In [ ]:
# Drop columns with ANY missing value
titanic_cleaned_cols = titanic.dropna(axis=1)
titanic_cleaned_cols.columns

In [ ]:
# Drop rows if  the 'age' has missing values
titanic_subset_cleaned = titanic.dropna(subset=['age'])
len(titanic_subset_cleaned)

### Option 2: Filling Missing Values (Imputation)

A better strategy is often to fill, or *impute*, the missing values. `df.fillna()` allows you to replace `NaN` with a statistical value like the mean, median, or mode.

Let's fill the missing `Age` values with the median age of all passengers. We use the median because it's less sensitive to extremely high or low ages (outliers) than the mean.

In [ ]:
# Create a copy to avoid changing our original DataFrame
titanic_filled = titanic.copy()

# Calculate the median age
median_age = titanic_filled['age'].median()
print(f"The median passenger age is: {median_age:.2f}")

We can fill missing 'age' values with the median using the `inplace=True` argument, which modifies the dataframe directly, without first assigning it to a new variable.

In [ ]:
titanic_filled['age'].fillna(median_age, inplace=True)

print("Missing values count after filling age:")
print(titanic_filled.isnull().sum())

___
## 💪 **Exercise** 💪

1. Find the most common port of embarkation.
2. On the `titanic_filled` DataFrame, fill the missing `embarked` values with the mode you found.
3. Print the `isnull().sum()` again to confirm the missing `embarked` NaNs are gone.
---

## 📓 Reflection 📓

Data cleaning is one of the more frustrating, yet important parts of working with authentic datasets. Do you think it's worth asking your students to do it? Or is it best to provide them pre-cleaned data?